# Základy měření s termokamerou

Na tomto cvičení budeme používat kamery [Tau2](https://www.flir.com/products/tau-2/) od firmy FLIR. 

Tau2 je schopno několika módů snímání. V prvním módu snímá veličinu, která je lineární s radiometrickým fluxem.
V druhém módu (který budeme používat my) snímá veličinu lineární s teplotou scény. Výhoda druhého modelu je taková, že kompenzuje vliv teploty jádra na naměřenou teplotu scény.

Kamery snímají infračervené záření o vlnové délce ~7.5 - ~13.5 μm, jinými slovy měří [radiometrický flux](https://en.wikipedia.org/wiki/Radiant_flux). Vztah mezi fluxem a zdánlivou povrchovou teplotou je popsaný ve funkcích `flux_to_temp` a `temp_to_flux`. Pro jednoduchost teplotu měříme v Kelvinech.

Parametry `R B F O` jsou fitnuté při kalibraci kamery, pro nás nemají žádný význam (pro zájemce [Planck's law](https://en.wikipedia.org/wiki/Planck%27s_law)).

Popíšeme model, který kamera používá, aby z celkového naměřeného fluxu (scene flux) vyrobila povrchovou teplotu měřeného objektu.
Kamera snímá celkový flux scény, ten se skládá z:
- flux vyzářený měřeným objektem
- flux vyzářený dalšími objekty ve scéně

Měřený flux je dále ovlivněn atmosférou, která část pohltí a část sama vyzáří.
V poslední řadě se aplikuje vliv filtru, který část záření pohltí a část odrazí.
Celkový flux, který naměří kamera nazýváme scene flux.

Kamera provádí postprocessing, aby ze scene flux vyrobila flux měřeného objektu.
Model si vyrobíme na tomto cvičení.

Platí:
$$
1 = \tau + r + \varepsilon,
$$
kde $\tau$ je transmisivita (propustnost), $r$ je reflexivita (odrazivost) a $\varepsilon$ je emisivita. 

Známé hodnoty emisivity (7.5 - 13.5 μm):
https://www.engineeringtoolbox.com/emissivity-coefficients-d_447.html

Známé hodnoty transmisivity (7.5 - 13.5 μm):
- germániové sklo: 0.4
- germániové sklo s antireflexním coatingem: 0.95

![Radiometrický model](./images/radiometric_model.png)

## Měření propustnosti fólie (2b)

In [3]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

In [4]:
def flux_to_temp(val):
    return (
        sensor_params.B / np.log(sensor_params.R / 
        (val - sensor_params.O) + sensor_params.F)
    )


def temp_to_flux(val):
    return (
        sensor_params.R / 
        (np.exp(sensor_params.B / val) - sensor_params.F) + sensor_params.O
    )


def tlin_to_temp(val):
    return val * 0.04


def atmospheric_transmissivity(atm_temp_C, humidity, distance_m):
    K1 = 1.5587e+0
    K2 = 6.9390e-2
    K3 = -2.7816e-4
    K4 = 6.8455e-7

    # alpha = 0.01262
    alpha = 0
    beta = -0.00667

    # NOTE: Tayloruv polynom 3. radu
    exponent = (
        K1 + K2 * atm_temp_C + 
        K3 * atm_temp_C**2 + K4 * atm_temp_C**3
    )

    sqrt_H2O = np.sqrt(humidity * np.exp(exponent))
    sqrt_distance_m = np.sqrt(distance_m)
    return np.exp(sqrt_distance_m * (alpha + beta * sqrt_H2O))

In [1]:
# Pozor, data získaná z kamery jsou 16bitová
imgs = {
    ...
}

## Zobrazte získaná data ve stupních C

In [ ]:
# TODO: implement

## Vykreslete graf závislosti transmisivity atmosféry na vzdálenosti a na vlhkosti

Použijte funkci atmospheric_transmisivity.

In [ ]:
# TODO: implement

## Změřte propustnost folie

Naimplementujte funkce, které budou sloužit na výpočet `obj_flux` a `scene_flux`.
Dále změřte propustnost filtru na objektivu a vytvořte model, který bude kompenzovat vliv filtru na výsledných snímcích.

In [8]:
class SensorParameters:
    R = 395654
    B = 1428
    F = 1
    O = 156

sensor_params = SensorParameters()

class RadiometricModel:
    obj_trans = 0  # zanedbáváme
    atm_reflex = 0  # zanedbáváme
    win_emis = 0  # zanedbáváme

    def __init__(self, *, obj_emis=0.95, bkg_temp=293.15,
            atm_temp=293.15, atm_trans=0.95, 
            win_temp=293.15, win_trans=0.95):
        # object
        self.obj_emis = obj_emis
        self.obj_reflex = 1 - obj_emis

        # background
        self.bkg_temp = bkg_temp

        # atmosphere
        self.atm_temp = atm_temp  
        self.atm_trans = atm_trans
        self.atm_emis = 1 - atm_trans

        # window
        self.win_temp = win_temp
        self.win_trans = win_trans
        self.win_reflex = 1 - win_trans

In [9]:
def obj_flux_to_scene_flux(model: RadiometricModel, obj_flux):
    # TODO: implement
    pass

def scene_flux_to_obj_flux(model: RadiometricModel, scene_flux):
    # TODO: implement
    pass

In [ ]:
# TODO: implement 